# End to End Pure Streaming Data-Pipeline for Landlord Table Using Spark Structured Streaming on Databricks

###### Description: In this notebook we read landlord state rows from incoming csv files into a streamig dataframe, transform (clean, cast, rename) the data, add/update the latest state to a Databricks Delta table
###### Objective: (incoming csv files) --> "landlord_streamingDF" --> "results_df" --> "landlord_data"

In [3]:
import requests
import json
import optimus as op
import phonenumbers 
import re
import datetime
import time

from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.functions import rank, col

In [4]:
# Schema for Landlord JSON
landlord_schema = StructType([
            StructField("Landlord_id", IntegerType(), False),
            StructField("Password", StringType(), True),
            StructField("Landlord_name", StringType(), False),
            StructField("Address_line_1", StringType(), False),
            StructField("City", StringType(), False),
            StructField("Post_code", StringType(), True),
            StructField("Region", StringType(), True),
            StructField("event_time", TimestampType(), True)])

landlord_udf_schema = StructType([
            StructField("Password", StringType(), True),
            StructField("Landlord_name", StringType(), False),
            StructField("Address_line_1", StringType(), False),
            StructField("City", StringType(), False),
            StructField("Post_code", StringType(), True),
            StructField("Region", StringType(), True),
            StructField("event_time", TimestampType(), True)])

###### Description: Get landlord csv files as a streaming "landlord_streamingDF" and process it on the fly and get transformed stream "landlord_df"
###### Objective: (incoming csv files) --> "landlord_streamingDF" --> "landlord_df"

In [6]:
# Get landlord Steaming DataFrame from csv files

# streaming starts here by reading the input files 
landlord_Path = "/FileStore/apartment/landlord/inprogress/"
landlord_streamingDF = (
  spark
    .readStream
    .schema(landlord_schema)
    .option("maxFilesPerTrigger", "1")
    .option("header", "true")
    .option("multiLine", "true")
    .csv(landlord_Path)
)
# Clear invalid rows
landlord_df = landlord_streamingDF.select("*").where("Landlord_id IS NOT NULL")
# Instantiation of DataTransformer class:
transformer = op.DataFrameTransformer(landlord_df)
# Replace NA with 0's
transformer.replace_na(0.0, columns="*")
# Clear accents: clear_accents only from name column and not everywhere 
transformer.clear_accents(columns='*')
# Remove special characters:  From all Columns 
transformer.remove_special_chars(columns=['Address_line_1', 'City', 'Post_code', 'Region'])

##### This function parses the corresponding columns into a single column

In [8]:
def my_fun(Password, Landlord_name, Address_line_1, City, Post_code, Region, event_time):
  return zip(Password, Landlord_name, Address_line_1, City, Post_code, Region, event_time)

udf_Fun = udf(my_fun, ArrayType(landlord_udf_schema))

In [9]:
Landlord_id, Password, Landlord_name, Address_line_1, City, Post_code, Region, event_time

In [10]:
intermediate_df = ( landlord_df.withWatermark("event_time", "10 seconds")
            .groupBy("Landlord_id")
            .agg(F.collect_list("Password").alias("Password"),
                 F.collect_list("Landlord_name").alias("Landlord_name"),
                 F.collect_list("Address_line_1").alias("Address_line_1"),
                 F.collect_list("City").alias("City"),
                 F.collect_list("Post_code").alias("Post_code"),
                 F.collect_list("Region").alias("Region"),
                 F.collect_list("event_time").alias("event_time"), 
                 F.max("event_time").alias("latest_event_time"))
            .select("Landlord_id", 
                    F.explode(udf_Fun(F.column("Password"), 
                                      F.column("Landlord_name"), 
                                      F.column("Address_line_1"), 
                                      F.column("City"), 
                                      F.column("Post_code"),
                                      F.column("Region"), 
                                      F.column("event_time")))
                    .alias("data"), "latest_event_time"))

##### Filter the data where event_time is latest

In [12]:
results_df = (intermediate_df
              .select("Landlord_id", 
                      "data.Password", 
                      "data.Landlord_name", 
                      "data.Address_line_1", 
                      "data.City", 
                      "data.Post_code", 
                      "data.Region",
                      "data.event_time", 
                      "latest_event_time")
              .where("data.event_time=latest_event_time")).orderBy("Landlord_id")

##### Display final result
###### This result shows the latest state of all the unique Building_id

In [14]:
display(results_df)

##### Below cells are optional if external functionality or storage is needed

###### Write the stream to a Databricks Delta table for storage

In [17]:
streaming_query = (results_df.writeStream
 .format("delta")
 .outputMode("complete")
 .option("mergeSchema", "true")
 .option("checkpointLocation", "/delta/apartment/landlord/_checkpoints/streaming-agg")
 .start("/delta/apartment/landlord_data"))

#### Read the Delta Table as a Static or Streaming DataFrame
#### This dataframe wil always be Up-To-Date

In [19]:
landlord_data = spark.read.format("delta").load("/delta/apartment/landlord_data").orderBy("Landlord_id")

In [20]:
display(landlord_data)

### Do Some Live Streaming Graphs

In [22]:
landlord_data_stream = spark.readStream.format("delta").load("/delta/apartment/landlord_data")

In [23]:
display(landlord_data_stream.groupBy("Region").count())